## Tiling of full images into overlapping patches

In [1]:
import numpy as np
from PIL import Image
import random
import matplotlib.pyplot as plt
import os
from tifffile import tifffile



def extract_overlapping_patches(image, patch_size=600, overlap=0.3):
    """
    Extracts overlapping patches of a given size from an image.
    
    :param image: Numpy array of the image.
    :param patch_size: The size of the square patches.
    :param overlap: Fraction of the patch size to overlap (e.g., 0.5 for 50% overlap).
    :return: List of image patches.
    """
    stride = int(patch_size * (1 - overlap))
    patches = []
    
    for y in range(0, image.shape[0] - patch_size + 1, stride):
        for x in range(0, image.shape[1] - patch_size + 1, stride):
            patch = image[y:y+patch_size, x:x+patch_size]
            patches.append(patch)
            
    return patches


In [ ]:
from skimage.restoration import denoise_nl_means, estimate_sigma
import numpy as np

def denoise_patch(patch):
    # Assuming patch is a 2D numpy array (grayscale). For RGB, add the channel dimension as needed.
    sigma_est = np.mean(estimate_sigma(patch))
    patch_kw = dict(patch_size=5,      # 5x5 patches
                    patch_distance=6)  # Search up to 6 pixels away for similar patches
    denoised_patch = denoise_nl_means(patch, h=1.15 * sigma_est, fast_mode=True, **patch_kw)
    return denoised_patch


In [2]:
import numpy as np

def normalize_patch(patch):
    """Normalize image patch to the range [0, 1]."""
    patch_min = patch.min()
    patch_max = patch.max()
    # Check if all pixel values are the same
    if patch_max == patch_min:

        normalized_patch = np.zeros_like(patch)
    else:
        normalized_patch = (patch - patch_min) / (patch_max - patch_min)
    return normalized_patch




In [6]:
import os
#denoised_patches = [denoise_patch(patch) for patch in patches]

image_filenames = ['data\PBS\Akita_22_OE_25x_stack_6.tif', 'data\PBS\Akita_22_OD_25x_stack_8.tif', 'data\FG12\Akita_23_OD_25x_stack_4.tif', 'data\FG12\Akita_2_OE_25x_stack_7.tif']

patch_size=512
overlap=0.3


for filename in image_filenames:
    image = tifffile.imread(filename)  
    patches = extract_overlapping_patches(image, patch_size, overlap)  
    normalized_patches = [normalize_patch(patch) for patch in patches]  

    # Extract the base filename without extension
    base_filename = os.path.basename(filename).split('.')[0]

    for i, patch in enumerate(normalized_patches):
        patch_image = Image.fromarray((patch * 255).astype(np.uint8))  
        # Save the image with the desired filename
        patch_image.save(f'data patch/{base_filename}_patch_{i}.png', 'PNG')



# Annotation selection

In [3]:
import os
import random
import shutil

def select_random_patches(source_folder, target_folder, num_patches):
    """
    Randomly selects a subset of patches for annotation.

    :param source_folder: The directory where all patches are stored.
    :param target_folder: The directory to store the selected patches for annotation.
    :param num_patches: The number of patches to randomly select.
    """
    os.makedirs(target_folder, exist_ok=True)
    
    all_patches = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    
    # Randomly select 'num_patches' patches
    selected_patches = random.sample(all_patches, num_patches)
    
    # Copy the selected patches to the target folder
    for patch in selected_patches:
        shutil.copy(os.path.join(source_folder, patch), os.path.join(target_folder, patch))
        
    print(f"Selected {num_patches} patches have been copied to {target_folder}.")

source_folder = 'data patch'  
target_folder = 'data patch/annotations'  
num_patches = 500 

select_random_patches(source_folder, target_folder, num_patches)

Selected 500 patches have been copied to data patch/annotations.
